In [1]:
# DOWNLOADING DATA

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

qjuices = pd.read_csv('Dataframes/qjuices.csv') # or index_col = 0 instead index = False in 'qjuices'

qjuices['date'] = pd.to_datetime(qjuices['date'])

qjuices

,year,date,weekday,holidays,id_store,id_brand,id_sku,salesvolume,av_price,name_taste,sizecategory,size
0,2009,2009-01-01,Thursday,Новый год,7,2019,10826,1,71,mix,2,1000
1,2009,2009-01-01,Thursday,Новый год,12,2019,10827,1,71,other,2,1000
2,2009,2009-01-01,Thursday,Новый год,7,2019,10827,2,71,other,2,1000
3,2009,2009-01-01,Thursday,Новый год,12,2019,10828,0,71,other,2,1000
4,2009,2009-01-01,Thursday,Новый год,7,2019,10828,6,71,other,2,1000
...,...,...,...,...,...,...,...,...,...,...,...,...
857291,2014,2014-12-31,Wednesday,Новый год,7,2018,1906980,13,69,orange,2,1000
857292,2014,2014-12-31,Wednesday,Новый год,12,2018,1906981,1,85,pineapple,2,1000
857293,2014,2014-12-31,Wednesday,Новый год,7,2018,1906981,2,85,pineapple,2,1000
857294,2014,2014-12-31,Wednesday,Новый год,12,2018,1907577,1,98,grapefruit,2,1000


In [2]:
qjuices[qjuices['name_taste'] == 'tomato'][qjuices['id_store'] == 7][['date', 'id_sku', 'salesvolume']].to_csv('Dataframes/tomato_distribution.csv', index = False)

In [3]:
# QUERYING AN APPROPRIATE DATA

tomato_7 = qjuices.query('id_store == 7')[qjuices['name_taste'] == 'tomato']\
    .drop(['year', 'id_brand', 'id_store', 'name_taste', 'size', 'sizecategory'], axis = 1)\
    .drop(['weekday', 'holidays'], axis = 1)\
    .reset_index(drop = True)

tomato_7['salesvolume'] = [sale if sale >= 0 else 0 for sale in tomato_7['salesvolume']]

# tomato_7

In [4]:
# SELECTING NOT ONE-TIME ITEMS

items_1 = tomato_7.groupby('id_sku')['salesvolume'].agg('sum').reset_index(False)
selected_items_1 = [i for i in items_1[items_1['salesvolume'] >= 600]['id_sku']]
tomato_7 = tomato_7[tomato_7['id_sku'].isin(selected_items_1)]

# items_2 = tomato_7[tomato_7['date'] > '2013-10-01'].groupby('id_sku')['salesvolume'].agg('sum').reset_index(False)
# selected_items_2 = [i for i in items_2[items_2['salesvolume'] >= 500]['id_sku']]
# tomato_7 = tomato_7[tomato_7['id_sku'].isin(selected_items_2)]
#
# tomato_7

In [5]:
# CHANGING TABLE STRUCTURE

tomato_7 = tomato_7.pivot(values = ['salesvolume', 'av_price'], index = 'date', columns = 'id_sku')

tomato_7.columns = ["_".join([pair[0], str(pair[1])]) for pair in tomato_7.columns]

tomato_7.reset_index(drop = False, inplace = True)

# tomato_7

In [6]:
# ADDING IS_SUPPLIED

for col in [col for col in tomato_7.columns if 'salesvolume' in col]:
    tomato_7['is_supplied_' + col.replace('salesvolume_', '')] = [1 if x >= 0 else 0 for x in tomato_7[col]]

tomato_7 = tomato_7.fillna(0)

# tomato_7

In [7]:
# CPI

CPI = pd.read_excel('Dataframes/CPI for Fruit juices, Perm Krai, in % to the previous month.xlsx', na_values = '?')

CPI = pd.melt(CPI, id_vars = ['Month'], var_name = 'Year', value_name = 'Value')\
    .dropna(subset = ['Value'])

CPI['Date'] = CPI['Month'] + '-' + CPI['Year'].astype('str')
CPI['Date'] = pd.to_datetime(CPI['Date'])

CPI = CPI[CPI['Year'] >= 2009]
CPI.reset_index(inplace = True, drop = True)

CPI.set_index('Date', inplace = True)
CPI.drop(['Month', 'Year'], axis = 1, inplace = True)

dates = pd.date_range(CPI.index.min() - pd.DateOffset(day=1), CPI.index.max() + pd.DateOffset(day=31), freq='D', name = 'date')
CPI = CPI.reindex(dates)\
    .fillna(100)\
    .astype('float')

def cummult(series):
    A = [1.0]
    for i in range(1, len(series)):
        A.append(A[i - 1] * series[i])
    return A

CPI['cpi'] = cummult(CPI['Value'].apply(lambda x: x / 100))

CPI.drop('Value', axis = 1, inplace = True)
CPI.reset_index(inplace = True)

# CPI[CPI['date'] == '2014-12-31']
CPI

,date,cpi
0,2009-01-01,1.000000
1,2009-01-02,1.000000
2,2009-01-03,1.000000
3,2009-01-04,1.000000
4,2009-01-05,1.000000
...,...,...
4802,2022-02-24,2.502913
4803,2022-02-25,2.502913
4804,2022-02-26,2.502913
4805,2022-02-27,2.502913


In [8]:
# ADJUSTING OF INFLATION

tomato_7 = tomato_7.merge(CPI, on = 'date')

for col in [col for col in tomato_7.columns if 'av_price_' in col]:
    tomato_7[col] = tomato_7[col] / tomato_7['cpi']

tomato_7.drop('cpi', axis = 1, inplace = True)

for col in [col for col in tomato_7.columns if 'av_price_' in col]:
    tomato_7[col] = tomato_7[col].replace(0, np.NaN)\
    .fillna(tomato_7[col].replace(0, np.NaN).median())
        # .fillna(method = 'ffill')\
        # .fillna(method = 'bfill')

# tomato_7

In [9]:
# CALCULATING DISCOUNT DATAFRAME I

discount = pd.DataFrame(tomato_7['date'])

for col in [col for col in tomato_7.columns if 'av_price_' in col]:
    med = tomato_7[col].median()
    disc = tomato_7[col].apply(lambda x: 0 if (med - x) / med < 0.25 else (med - x) / med)
    if sum(disc) > 0.25 * 20:
        discount['is_discount_' + col.replace('av_price_', '')] = disc

discount

,date,is_discount_46135,is_discount_46902,is_discount_59042,is_discount_75320,is_discount_201676,is_discount_362058,is_discount_415514,is_discount_415824,is_discount_419020,...,is_discount_459894,is_discount_459896,is_discount_461504,is_discount_468496,is_discount_468558,is_discount_471503,is_discount_481677,is_discount_489135,is_discount_495154,is_discount_600761
0,2009-01-01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,2009-01-02,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,2009-01-03,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,2009-01-04,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,2009-01-05,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2014-12-27,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297909,0.396074,0.0
2187,2014-12-28,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297909,0.396074,0.0
2188,2014-12-29,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297909,0.396074,0.0
2189,2014-12-30,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297909,0.396074,0.0


In [10]:
# SELECTING ITEMS WHICH HAVE DISCOUNT

IDs = [col.strip('is_discount_') for col in discount.columns if 'is_discount_' in col]

selected_items_2 = []
for name in ['av_price_', 'salesvolume_', 'is_supplied_']:
    selected_items_2 += [col for col in tomato_7.columns if col.replace(name, '') in IDs]

tomato_7 = tomato_7[['date'] + selected_items_2]
tomato_7

,date,av_price_46135,av_price_46902,av_price_59042,av_price_75320,av_price_201676,av_price_362058,av_price_415514,av_price_415824,av_price_419020,...,is_supplied_459894,is_supplied_459896,is_supplied_461504,is_supplied_468496,is_supplied_468558,is_supplied_471503,is_supplied_481677,is_supplied_489135,is_supplied_495154,is_supplied_600761
0,2009-01-01,65.000000,47.000000,56.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
1,2009-01-02,65.000000,48.000000,60.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
2,2009-01-03,66.000000,48.000000,61.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
3,2009-01-04,65.000000,48.000000,61.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
4,2009-01-05,66.000000,48.000000,61.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2014-12-27,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,52.483479,...,0,0,0,1,1,1,0,1,1,1
2187,2014-12-28,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,55.174940,...,0,0,0,1,1,1,0,1,1,1
2188,2014-12-29,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,55.174940,...,0,0,0,1,1,1,0,1,1,1
2189,2014-12-30,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,55.174940,...,0,0,0,1,1,1,0,1,1,1


In [11]:
# # CALCULATING DISCOUNT DATAFRAME I
#
# discount = pd.DataFrame(tomato_7['date'])
#
# for col in [col for col in tomato_7.columns if 'av_price_' in col]:
#     disc_list = []
#
#     for i, price in enumerate(tomato_7[col]):
#
#         if i != 0:
#             margin_backward = (tomato_7[col][i - 1] - price) / tomato_7[col][i - 1]
#             margin_forward = (price - tomato_7[col][i - 1]) / tomato_7[col][i - 1]
#
#         if tomato_7['is_supplied_' + col.replace('av_price_', '')][i] == 0 or i == 0:
#             disc_list.append(0)
#
#         elif margin_backward >= 0.2:
#             disc_list.append(margin_backward)   # 1
#
#         elif disc_list[i - 1] != 0 and margin_forward <= 0.2:
#             disc_list.append(disc_list[i - 1])  # 1
#
#         elif disc_list[i - 1] != 0 and margin_forward >= 0.2:
#             disc_list.append(0)
#
#         elif disc_list[i - 1] == 0 and (abs(margin_backward) <= 0.2 or price > tomato_7[col][i - 1]):
#             disc_list.append(0)
#
#     if sum(disc_list) > 5 * 0.2:
#         discount['is_discount_' + col.replace('av_price_', '')] = disc_list
#
# discount

In [12]:
# CALCULATING DISCOUNT DATAFRAME II

# for id in IDs:
#     days_with_discount_list = []
#     for i, val in enumerate(discount['is_discount_' + id]):
#         if val == 0:
#             days_with_discount_list.append(0)
#         else:
#             days_with_discount_list.append(days_with_discount_list[i - 1] + 1 if i > 0 else 1)
#
#     discount['num_days_discount_' + id] = days_with_discount_list
#
# for id in IDs:
#     days_without_discount_list = []
#     for i, val in enumerate(discount['is_discount_' + id]):
#         if val == 0:
#             days_without_discount_list.append((days_without_discount_list[i - 1] + 1) if i > 0 else 1)
#         else:
#             days_without_discount_list.append(0)
#
#     discount['num_days_without_discount_' + id] = days_without_discount_list

for id in IDs:
    other_dics_list = []
    for i in range(0, len(discount['is_discount_' + id])):
        other_dics_list.append(int(sum(
            [1 if val > 0 else 0 for val in discount[[col for col in discount.columns if 'is_discount_' in col]].loc[i, :]]
        )))
    discount['num_other_discounts_' + id] = other_dics_list

discount

,date,is_discount_46135,is_discount_46902,is_discount_59042,is_discount_75320,is_discount_201676,is_discount_362058,is_discount_415514,is_discount_415824,is_discount_419020,...,num_other_discounts_459894,num_other_discounts_459896,num_other_discounts_461504,num_other_discounts_468496,num_other_discounts_468558,num_other_discounts_471503,num_other_discounts_481677,num_other_discounts_489135,num_other_discounts_495154,num_other_discounts_600761
0,2009-01-01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2009-01-02,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2009-01-03,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2009-01-04,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2009-01-05,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2014-12-27,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,5,5,5,5,5,5,5,5,5,5
2187,2014-12-28,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,5,5,5,5,5,5,5,5,5,5
2188,2014-12-29,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,5,5,5,5,5,5,5,5,5,5
2189,2014-12-30,0.304664,0.0,0.0,0.387326,0.0,0.0,0.358436,0.0,0.0,...,5,5,5,5,5,5,5,5,5,5


In [13]:
# ADDING DUMMIES: HOLIDAYS, WEEKDAYS, MONTH

import holidays

tomato_7['weekday'] = tomato_7['date'].apply(lambda x: x.strftime('%A'))
tomato_7['month'] = tomato_7['date'].apply(lambda x: x.strftime('%B'))

tomato_7 = pd.get_dummies(tomato_7, drop_first = False)\
    .drop(['weekday_Monday',
           'month_January'], axis = 1)

# from deep_translator import GoogleTranslator
# import functools
#
# @functools.cache
# def translator(x, source = 'russian', target = 'english'):
#     return GoogleTranslator(source = source, target = target).translate(x)
#
# tomato_7['holidays'] = tomato_7['date'].apply(lambda x: holidays.RU().get(x))\
#     .replace({None:'Regular day'})\
#     .apply(lambda x: translator(x))\
#     .apply(lambda x: x.replace("'", '_').replace(' ', '_'))
#
# tomato_7['holidays_forward'] = tomato_7['holidays'].shift(-1)
# tomato_7 = tomato_7[:-1]
#
# tomato_7.reset_index(drop = True, inplace = True)
#
# tomato_7 = pd.get_dummies(tomato_7, drop_first = False)\
#     .drop(['holidays_Regular_day',
#            'holidays_forward_Regular_day'], axis = 1)

discount = discount[discount['date'] >= min(tomato_7['date'])].reset_index(drop = True)
tomato_7

,date,av_price_46135,av_price_46902,av_price_59042,av_price_75320,av_price_201676,av_price_362058,av_price_415514,av_price_415824,av_price_419020,...,month_August,month_December,month_February,month_July,month_June,month_March,month_May,month_November,month_October,month_September
0,2009-01-01,65.000000,47.000000,56.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
1,2009-01-02,65.000000,48.000000,60.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
2,2009-01-03,66.000000,48.000000,61.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
3,2009-01-04,65.000000,48.000000,61.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
4,2009-01-05,66.000000,48.000000,61.000000,84.000000,42.000000,49.943062,36.707605,69.01367,59.973953,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2014-12-27,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,52.483479,...,0,1,0,0,0,0,0,0,0,0
2187,2014-12-28,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,55.174940,...,0,1,0,0,0,0,0,0,0,0
2188,2014-12-29,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,55.174940,...,0,1,0,0,0,0,0,0,0,0
2189,2014-12-30,46.427693,50.434773,59.985598,50.464884,37.007582,49.943062,23.550279,69.01367,55.174940,...,0,1,0,0,0,0,0,0,0,0


In [14]:
tomato_7.drop([c for c in tomato_7.columns if IDs[-2] in c], axis = 1, inplace = True)
discount.drop([c for c in discount.columns if IDs[-2] in c], axis = 1, inplace = True)

In [15]:
# SAVING

tomato_7.to_csv('Dataframes/tomato_7.csv', index = False)

discount.to_csv('Dataframes/tomato_discount.csv', index = False)